# 第7回講義 宿題

## 課題. 変分オートエンコーダ（VAE）でFasionMNISTの画像を生成せよ


### ルール

- 訓練データはx_train、テストデータはx_testで与えられます。
- 下のセルで指定されているx_train以外の学習データは使わないでください。
- 提出コードの一番上の部分に工夫した事柄を数行でコメントアウトして書いてください（もっと書きたい人はたくさん書いても大丈夫です）。

### 目標値

NLL（負の対数尤度） 235

### 提出方法

- 2つのファイルを提出していただきます。
  - テストデータ (x_test) に対する予測ラベルをcsvファイル (ファイル名: submission_pred.csv) で提出してください。
  - それに対応するpythonのコードをsubmission_code.pyとして提出してください (%%writefileコマンドなどを利用してください)。

### 評価方法

- 評価は生成画像のテストデータに対するNLL（負の対数尤度）で行います.$-\sum_{i=1}^Dx_i\log\hat{x_i}+(1-x_i)\log(1-\hat{x_i})$
- 定時にNLLを計算しLeader Boardを更新します。
- 締切後のNLLを最終的な評価とします

### データの読み込み

- この部分は修正しないでください

In [27]:
import numpy as np
import pandas as pd
import torch
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm
from PIL import Image
from sklearn.model_selection import train_test_split

# # 学習データ
# x_train = np.load('drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210527/data/x_train.npy')
    
# # テストデータ
# x_test = np.load('drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210527/data/x_test.npy')
#学習データ
x_train = np.load('/Users/明朗/workspace/deeplearningUT/Lecture07/data/x_train.npy')
    
#テストデータ
x_test = np.load('/Users/明朗/workspace/deeplearningUT/Lecture07/data/x_test.npy')


class dataset(torch.utils.data.Dataset):
    def __init__(self, x_train):
        data = x_train.astype('float32')
        self.x_train = []
        for i in range(data.shape[0]):
            self.x_train.append(Image.fromarray(np.uint8(data[i])))
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.x_train)

    def __getitem__(self, idx):
        return self.transform(self.x_train[idx])

trainval_data = dataset(x_train)
test_data = dataset(x_test)

### VAEの実装


In [28]:
batch_size = 32

val_size = 10000
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

In [51]:
'''
（工夫した事柄をこの部分に記入）
vaeの層をconvolutionで記述した．
beta-vaeも実装したが，そこまで性能の向上がみられなかったため，beta=1で学習した．
'''
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F

device = 'cuda'


# torch.log(0)によるnanを防ぐ
def torch_log(x):
    return torch.log(torch.clamp(x, min=1e-10))


class VAE(nn.Module):
    def __init__(self, z_dim):
        super(VAE, self).__init__()
        self.dense_enc1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), #28 28 1 -> 28 28 32
            nn.BatchNorm2d(32)
        )
        self.dense_enc2 = nn.Sequential(
            nn.Conv2d(32, 32, 3, padding=1), # 28 28 32 -> 28 28 32
            nn.BatchNorm2d(32)
        )
        self.pool = nn.MaxPool2d(2)         #28 28 32 -> 14 14 32
        self.flat = nn.Flatten()
        self.dense_encmean = nn.Linear(28*28*32, z_dim)
        self.dense_encvar = nn.Linear(28*28*32, z_dim)
        self.dense_dec1 = nn.Linear(z_dim, 28*28*32)
        self.dense_dec2 = nn.Sequential(
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32)
        )
        self.dense_dec3 = nn.Sequential(
            nn.Conv2d(32, 1, 3, padding=1),
            nn.BatchNorm2d(1)
        )

    def _encoder(self, x):
        x = F.relu(self.dense_enc1(x))
        x = F.relu(self.dense_enc2(x))
        #x = self.pool(x)
        x = self.flat(x)
        mean = self.dense_encmean(x)
        var = F.softplus(self.dense_encvar(x))
        return mean, var

    def _sample_z(self, mean, var):
        epsilon = torch.randn(mean.shape).to(device)
        return mean + torch.sqrt(var) * epsilon

    def _decoder(self, z):
        x = F.relu(self.dense_dec1(z))
        x = torch.reshape(x, (-1, 32, 28, 28))
        x = F.relu(self.dense_dec2(x))
        x = torch.sigmoid(self.dense_dec3(x))
        return x

    def forward(self, x):
        mean, var = self._encoder(x)
        z = self._sample_z(mean, var)
        x = self._decoder(z)
        return x, z

    def loss(self, x, beta=1):
        mean, var = self._encoder(x)
        # KL lossの計算
        KL = -0.5*torch.mean(torch.sum(1 + torch_log(var) - mean**2 - var, dim=1)) # WRITE ME

        z = self._sample_z(mean, var)
        y = self._decoder(z)

        # reconstruction lossの計算(今回はL=1)
        reconstruction = torch.mean(torch.sum(x*torch_log(y) + (1-x)*torch_log(1-y), dim=1)) # WRITE ME

        return beta*KL, -reconstruction

In [59]:
z_dim = 10
n_epochs = 30
model = VAE(z_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(n_epochs):
    losses = []
    KL_losses = []
    reconstruction_losses = []
    model.train()
    for x in dataloader_train:

        # WRITE ME
        x = x.to(device)
        model.zero_grad()
        y = model(x)
        KL_loss, reconstruction_loss = model.loss(x, 4)
        loss = KL_loss + reconstruction_loss
        loss.backward()
        optimizer.step()

        losses.append(loss.cpu().detach().numpy())
        KL_losses.append(KL_loss.cpu().detach().numpy())
        reconstruction_losses.append(reconstruction_loss.cpu().detach().numpy())

    losses_val = []
    model.eval()
    for x in dataloader_valid:

        # WRITE ME
        x = x.to(device)
        y = model(x)
        KL_loss, reconstruction_loss = model.loss(x, 4)
        loss = KL_loss + reconstruction_loss

        losses_val.append(loss.cpu().detach().numpy())

    print('EPOCH:%d, Train Lower Bound:%lf, (%lf, %lf), Valid Lower Bound:%lf' %
          (epoch+1, np.average(losses), np.average(KL_losses), np.average(reconstruction_losses), np.average(losses_val)))

EPOCH:1, Train Lower Bound:4.223601, (3.694654, 0.528947), Valid Lower Bound:1.390561
EPOCH:2, Train Lower Bound:0.786652, (0.293958, 0.492694), Valid Lower Bound:0.630111
EPOCH:3, Train Lower Bound:0.570477, (0.079111, 0.491365), Valid Lower Bound:0.557828
EPOCH:4, Train Lower Bound:0.512392, (0.021280, 0.491112), Valid Lower Bound:0.500929
EPOCH:5, Train Lower Bound:0.496155, (0.005198, 0.490957), Valid Lower Bound:0.491953
EPOCH:6, Train Lower Bound:0.492869, (0.001937, 0.490932), Valid Lower Bound:0.491425
EPOCH:7, Train Lower Bound:0.492128, (0.001283, 0.490845), Valid Lower Bound:0.492238
EPOCH:8, Train Lower Bound:0.491770, (0.000985, 0.490786), Valid Lower Bound:0.490911
EPOCH:9, Train Lower Bound:0.491454, (0.000680, 0.490773), Valid Lower Bound:0.490280
EPOCH:10, Train Lower Bound:0.491022, (0.000277, 0.490745), Valid Lower Bound:0.490172
EPOCH:11, Train Lower Bound:0.490856, (0.000137, 0.490719), Valid Lower Bound:0.490034
EPOCH:12, Train Lower Bound:0.494906, (0.004208, 0.4

In [53]:
import csv

sample_x = []
answer = []
model.eval()
for x in dataloader_test:

    x = x.to(device)

    y, _ = model(x)

    y = y.tolist()

    sample_x.extend(y)

import os
os.makedirs('output', exist_ok=True)

#with open('drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210527/submission_pred.csv', 'w') as file:
with open('output/submission_pred.csv', 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(sample_x)
file.close()